# Parallelized Data Generation with GRID Enterprise

This notebook demonstrates how to parallelize and scale data generation across your cluster using GRID Enterprise.

This assumes you have installed all the dependencies and have initiated the sessions on your cluster.

For more information, please visit [GRID Enterprise Documentation](https://docs.scaledfoundations.ai/grid_enterprise/index.html).

In [9]:
# Import necessary modules
import os
import json
from typing import List
from grid.sdk.manager import GRIDSessionManager  # Adjust import path if needed

# Initialize GRIDSessionManager
manager = GRIDSessionManager()
# List nodes and store IPs in a list
nodes = manager.list_nodes()

Loading resource configuration from /home/grid/.grid/resource_config.json...
+-------------+---------------+
| Node Name   | IP Address    |
+=============+===============+
| local       | localhost     |
+-------------+---------------+
| node1       | 20.83.236.157 |
+-------------+---------------+


In [10]:
resource_config = manager.load_resource_config()
# Extract node information
nodes_info = []
for node_name, details in resource_config.get("resources", {}).items():
    node_info = {
        "node_name": node_name,
        "ip": details.get("ip"),
        "username": details.get("username", ""),  # Default to empty string if missing
        "password": details.get("password", "")   
    }
    nodes_info.append(node_info)


Loading resource configuration from /home/grid/.grid/resource_config.json...


# Data Generation for Multiple Environments

In this example, we aim to generate data using the recording API and a simple script across multiple environments. To achieve parallelized runs, we’ll first create session configurations for each environment we want to process concurrently.

## Steps

1. **Define the Environments:** 
   We specify a list of environments (`env_list`) that we want to include in our data generation, such as `"abandoned_factory"`, `"electric_central"`, and `"neighborhood"`. This list can be modified to include other environments as required.

2. **Set Up the Base Configuration Template:** 
   A base JSON structure (`base_json`) defines essential simulation settings, including:
   - The `airgen` section, which configures environment-specific settings such as the `SimMode`, `VehicleType`, and `VehicleModel`.
   - The `grid` section, which specifies entities for the simulation, including a robot named `"airgen-drone"`.

Each environment in `env_list` will use this base template, and we’ll modify `env_name` for each to parallelize data generation runs across the cluster.

In [11]:
# Define the list of environments
env_list = ["abandoned_factory", "electric_central", "neighborhood"]  # Replace with actual environment names

# Create the base JSON template
base_json = {
    "airgen": {
        "env_name": "neighborhood",
        "geo": False,
        "settings": {
            "SimMode": "Multirotor",
            "Vehicles": {
                "Drone": {
                    "VehicleType": "SimpleFlight",
                    "VehicleModel": "Default"
                }
            }
        }
    },
    "grid": {
        "entities": {
            "robot": [{"name": "airgen-drone", "kwargs": {}}],
            "model": []
        }
    }
}

In [12]:
# Create the 'sweep' folder if it doesn't already exist
sweep_folder = os.path.expanduser("~/sweep")
os.makedirs(sweep_folder, exist_ok=True)
print(f"Folder created for storing session configs: {sweep_folder}")

Folder created for storing session configs: /home/grid/sweep


# Creating Unique Session Configurations with Recording Setup

To set up parallelized data generation across multiple environments, we create unique session configurations for each environment and define a separate recording folder for each session.

## Process

1. **Session ID Counter:**
   We start with `session_id = 1` and increment it for each environment to ensure each session has a unique identifier.

2. **Environment Loop:**
   For each environment in `env_list`:
   - A copy of the base configuration template (`base_json`) is modified to set the environment name (`env_name`).
   - We add a unique recording configuration for each session, setting up a dedicated folder under `/tmp/recording/{session_id}/{env}` where data will be stored separately for each job.

3. **Write Config Files:**
   Each environment’s configuration is saved to a JSON file in the `sweep_folder` directory, ensuring each session has its own config file.

This setup enables us to generate and store data independently for each environment, making it easy to manage data output for parallel jobs.

In [13]:
# Start the session ID counter
session_id = 1

# Loop over each environment to create session configs
for env in env_list:
    # Copy the base JSON and update it with environment-specific settings
    config = base_json.copy()
    config["airgen"]["env_name"] = env
    
    # Add the recording configuration
    config["airgen"]["settings"]["Recording"] = {
        "RecordOnMove": False,
        "RecordInterval": 0.05,
        "Folder": f"/tmp/recording/{session_id}/{env}",
        "Enabled": False,
        "Cameras": [
            {
                "CameraName": "0",
                "ImageType": 0,
                "PixelsAsFloat": False,
                "VehicleName": "",
                "Compress": True
            }
        ]
    }
    
    # Write the configuration to a JSON file
    config_file_path = os.path.join(sweep_folder, f"session_{session_id}_{env}.json")
    with open(config_file_path, 'w') as f:
        json.dump(config, f, indent=4)
    
    print(f"Session config created: {config_file_path}")
    
    # Increment session ID
    session_id += 1

Session config created: /home/grid/sweep/session_1_abandoned_factory.json
Session config created: /home/grid/sweep/session_2_electric_central.json
Session config created: /home/grid/sweep/session_3_neighborhood.json


# Helper Functions

- **_upload_via_sftp_async:** Asynchronously uploads files or directories to either a local or remote node.
- **execute_command_async:** Executes a command asynchronously on a specified local or remote node.

In [14]:
import asyncio
import shlex
import asyncssh
import subprocess
import shutil
import os

async def _upload_via_sftp_async(node_info, local_path, node_name, remote_folder):
    """
    Asynchronously uploads a file via SFTP using asyncssh.
    Handles local file upload separately if node_name is 'local'.
    """
    if node_name == "local":
        # Ensure remote_folder exists locally
        os.makedirs(remote_folder, exist_ok=True)
        
        if os.path.isdir(local_path):
            # Copy only the contents of the directory
            for item in os.listdir(local_path):
                s = os.path.join(local_path, item)
                d = os.path.join(remote_folder, item)
                if os.path.isdir(s):
                    shutil.copytree(s, d, dirs_exist_ok=True)
                else:
                    shutil.copy2(s, d)
        else:
            # Copy single file directly
            shutil.copy2(local_path, remote_folder)
        
        print(f"Copied contents of {local_path} to {remote_folder} on local machine.")
    else:
        # Remote upload using SFTP
        remote_ip = node_info["ip"]
        username = node_info["username"]
        password = node_info["password"]

        try:
            async with asyncssh.connect(remote_ip, username=username, password=password) as conn:
                async with conn.start_sftp_client() as sftp:
                    try:
                        await sftp.mkdir(remote_folder)
                    except asyncssh.SFTPError:
                        pass  # Ignore error if folder already exists

                    # Upload files or folder contents
                    if os.path.isdir(local_path):
                        for root, _, files in os.walk(local_path):
                            remote_subfolder = os.path.join(remote_folder, os.path.relpath(root, local_path)).replace("\\", "/")
                            try:
                                await sftp.mkdir(remote_subfolder)
                            except asyncssh.SFTPError:
                                pass  # Ignore if folder already exists
                            for file in files:
                                local_file_path = os.path.join(root, file)
                                remote_file_path = os.path.join(remote_subfolder, file).replace("\\", "/")
                                await sftp.put(local_file_path, remote_file_path)
                    else:
                        remote_file_path = os.path.join(remote_folder, os.path.basename(local_path)).replace("\\", "/")
                        await sftp.put(local_path, remote_file_path)
                    print(f"Uploaded {local_path} to {node_name}:{remote_folder}")
        except asyncssh.SFTPError as e:
            print(f"Error uploading via SFTP to {node_name}: {e}")
        except Exception as e:
            print(f"General error uploading via SFTP to {node_name}: {e}")
        

async def execute_command_async(command: str, node_info: dict, run_quiet: bool = False) -> str:
    """Execute a command asynchronously on a local or remote node."""
    node_name = node_info.get("node_name")
    ip = node_info.get("ip")
    username = node_info.get("username")
    password = node_info.get("password")

    if node_name == "local":
        # Local execution using asyncio with subprocess
        process = await asyncio.create_subprocess_exec(
            *shlex.split(command),
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        stdout, stderr = await process.communicate()

        if stdout:
            if not run_quiet:
                print(stdout.decode().strip())
            return stdout.decode().strip()
        
        if stderr:
            error = stderr.decode().strip()
            print(f"Error: {error}")
            return f"Error: {error}"

    else:
        # Remote execution with asyncssh
        try:
            async with asyncssh.connect(ip, username=username, password=password) as conn:
                result = await conn.run(command, check=True)
                output = result.stdout.strip()
                
                if not run_quiet:
                    print(f"Output from {node_name}: {output}")
                
                return output

        except asyncssh.Error as e:
            error_msg = f"SSH Error on {node_name}: {str(e)}"
            print(error_msg)
            return error_msg
        except Exception as e:
            error_msg = f"Exception on {node_name}: {str(e)}"
            print(error_msg)
            return error_msg

# Parallel Data Generation Across Nodes

In this section, we create a job queue for each node to assign session configurations, distributing the data generation work across multiple nodes in parallel.

1. **Session Config Distribution**: The function begins by reading session config files and dividing them equally across the available nodes. This means each node receives a portion of the configuration files to process.

2. **Remote Setup and Execution**: 
   - For each node, we ensure a temporary folder (`/tmp/datagen_code/`) exists locally on the node, then upload the necessary code files. The code is then copied into the Docker container on that node.
   - A session is started for each configuration file on each node. We also add a recording folder specific to each session within the container to keep generated data organized.

3. **Parallel Execution**: Each node runs its assigned session configs sequentially. However, nodes themselves are working in parallel with other nodes to maximize throughput across the cluster.

4. **Running the Script and Pausing**:
   - After starting the session, a pause is added to ensure the session is fully initialized before proceeding.
   - Next, the main data generation script runs inside the container.
   - Finally, each session is stopped once completed, ensuring the node is ready for the next job.

This approach effectively parallelizes the data generation tasks, distributing them across all nodes in the cluster, enhancing speed and scalability.

In [15]:
async def datagen_parallel(manager, session_settings_path, node_list, code_folder):
    """Simplified parallel data generation across nodes with session configs."""

    # Load session config files and distribute them to nodes equally
    config_files = [file for file in os.listdir(session_settings_path) if file.endswith('.json') or file.endswith('.yml')]
    num_nodes = len(node_list)
    configs_per_node = {node['node_name']: config_files[i::num_nodes] for i, node in enumerate(node_list)}

    # Define remote paths
    local_tmp_folder = '/tmp/datagen_code/'
    docker_tmp_folder = '/tmp/datagen_tmp_code/'
    script_path = os.path.join('/tmp/datagen_tmp_code/datagen_code', 'drone_flight_test.py')
    
    async def process_node_sessions(node, configs):
        """Process each session config for a single node sequentially."""
        node_name = node["node_name"]
        node_ip = node["ip"]
        container_name = "grid_core"

        # Ensure local folder exists and upload code to node’s local tmp folder
        print(f"Preparing datagen code folder on {node_name}...")
        await execute_command_async(f"mkdir -p {local_tmp_folder}", node)
        await _upload_via_sftp_async(node,code_folder, node_name, local_tmp_folder)
        
        # Copy code to the container’s tmp directory
        print(f"Copying code folder to {docker_tmp_folder} in {container_name} on {node_name}...")
        await execute_command_async(f"docker cp {local_tmp_folder} {container_name}:{docker_tmp_folder}", node)

        for config_file in configs:
            session_id = os.path.splitext(config_file)[0]
            session_config_path = os.path.join(session_settings_path, config_file)

            # Load config and create recording folder in container
            config_data = manager.create_config(session_config_path, session_id)
            recording_folder = config_data["session"]["airgen"]["settings"]["Recording"]["Folder"]
            
            # Ensure recording folder exists in the container
            await execute_command_async(f"docker exec grid_service mkdir -p {recording_folder}", node)
            
            # Start the session
            print(f"Starting session {session_id} on {node_name}...")
            await manager.start_session(session_id=session_id, session_config_file_path=session_config_path, node_ip=node_ip)
            
            # Pause asynchronously for a specified duration (e.g., 5 seconds)
            pause_duration = 30  # Adjust the duration as needed
            print(f"Pausing for {pause_duration} seconds...")
            await asyncio.sleep(pause_duration)
    

            # Run the flight test script asynchronously in background
            print(f"Running script {script_path} for session {session_id} on {node_name}...")
            await execute_command_async(f"docker exec grid_core python {script_path}", node)
            
            # Stop the session
            print(f"Stopping session {session_id} on {node_name}...")
            await manager.stop_session(session_id=session_id)
    
    # Gather tasks for all nodes and execute them concurrently
    await asyncio.gather(*(process_node_sessions(node, configs_per_node[node["node_name"]]) for node in node_list))


After running the data generation process, you will see that recording folders for each session are created inside the `grid_service` container. These folders store the recording data specific to each session, allowing you to easily manage and access the output generated for each environment.

In [16]:
# Define the paths for the session settings and code folder
session_settings_path = os.path.expanduser("~/sweep")
code_folder = "/tmp/sample_code"  # Replace with your actual code folder path

# Now run `datagen_parallel`
await datagen_parallel(manager, session_settings_path, nodes_info, code_folder)

Preparing datagen code folder on local...
Preparing datagen code folder on node1...
Copied contents of /home/grid/airgenpractise/AirgenPractise/sample_code to /tmp/datagen_code/ on local machine.
Copying code folder to /tmp/datagen_tmp_code/ in grid_core on local...
Starting session session_1_abandoned_factory on local...
Starting session session_1_abandoned_factory ...


Output from node1: 
Uploaded /home/grid/airgenpractise/AirgenPractise/sample_code to node1:/tmp/datagen_code/
Copying code folder to /tmp/datagen_tmp_code/ in grid_core on node1...
Output from node1: 
Output from node1: 
Starting session session_2_electric_central on node1...
Starting session session_2_electric_central ...
response: Starting session...
response: Downloading resources..
response_end: Session has been started successfully
Session started successfully.
Pausing for 30 seconds...
response: Starting session...
response: Downloading resources..
response_end: Session has been started successfully
Session started successfully.
Pausing for 30 seconds...
Running script /tmp/datagen_tmp_code/datagen_code/drone_flight_test.py for session session_1_abandoned_factory on local...
Running script /tmp/datagen_tmp_code/datagen_code/drone_flight_test.py for session session_2_electric_central on node1...
Connecting to AirGen simulator...
Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (